In [1]:
import os
import pandas as pd
import numpy as np
from astropy import units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

VPath = '/Users/87steven/Documents/ASIAA/Blazar SED code and data/VLASS match'

Vname = []

allFileList = os.listdir(VPath)

for file in allFileList:
    
    if os.path.isfile(VPath+'//'+file):
        
        Vname.append(file)
        
    else:

        print('NO file exist !!')

#######################################################################################################
GPath = '/Users/87steven/Documents/ASIAA/Blazar SED code and data/GLEAM match'

Gname = []

allFileList = os.listdir(GPath)

for file in allFileList:
    
    if os.path.isfile(GPath+'//'+file):
        
        Gname.append(file)
        
    else:

        print('NO file exist !!')
        
#######################################################################################################
SPath = '/Users/87steven/Documents/ASIAA/Blazar SED code and data/SDSS match'

Sname = []

allFileList = os.listdir(SPath)

for file in allFileList:
    
    if os.path.isfile(SPath+'//'+file):
        
        Sname.append(file)
        
    else:

        print('NO file exist !!')

#######################################################################################################
WPath = '/Users/87steven/Documents/ASIAA/Blazar SED code and data/WISE match'

Wname = []

allFileList = os.listdir(WPath)

for file in allFileList:
    
    if os.path.isfile(WPath+'//'+file):
        
        Wname.append(file)
        
    else:

        print('NO file exist !!')

#######################################################################################################
FPath = '/Users/87steven/Documents/ASIAA/Blazar SED code and data/FIRST match'

Fname = []

allFileList = os.listdir(FPath)

for file in allFileList:
    
    if os.path.isfile(FPath+'//'+file):
        
        Wname.append(file)
        
    else:

        print('NO file exist !!')

def ClearData():
    data = {'Name': [],
    'RA': [],
    'DEC': [],
    'z': [], 
    'class_name': [],
    'alpha': [],
    'alphaRad': [],
    'alphaK': [],
    'alphaRadK': []
      }
    return data

In [ ]:
df = ClearData()

Xmatch = pd.read_csv('/Users/87steven/Documents/ASIAA/Blazar SED code and data/data/BZCAT_Xmatch_ACC_csv.csv') # # = 37671

#for i in range(0, 1): # 1367
for i, Dataname in enumerate(set(Xmatch.name[0: len(Xmatch.RA_hh)])):  # len(Xmatch.RA_hh)
    
    #Dataname = 'J2349+3849'
    plotfigure = 1
    
    VPath = '/Users/87steven/Documents/ASIAA/Blazar SED code and data/VLASS match'
    Vdata = pd.read_csv(VPath + '/' + Dataname + '.csv') 
    
    GPath = '/Users/87steven/Documents/ASIAA/Blazar SED code and data/GLEAM match'
    Gdata = pd.read_csv(GPath + '/' + Dataname + '.csv') 
    
    SPath = '/Users/87steven/Documents/ASIAA/Blazar SED code and data/SDSS match'
    Sdata = pd.read_csv(SPath + '/' + Dataname + '.csv') 
    
    WPath = '/Users/87steven/Documents/ASIAA/Blazar SED code and data/WISE match'
    Wdata = pd.read_csv(WPath + '/' + Dataname + '.csv')
    
    FPath = '/Users/87steven/Documents/ASIAA/Blazar SED code and data/FIRST match'
    Fdata = pd.read_csv(FPath + '/' + Dataname + '.csv')
    
    print('i = ', i, ', V source name = ', Dataname)
    
    #######################################################################################################
    ### extract flux density or magnitude 
    ## BZCAT
    Name = Vdata.Name[0] #Xmatch data
    RA = Vdata.RA[0]
    DEC = Vdata.DEC[0]
    z = Vdata.z[0]
    cla = Vdata['class'][0]
    flux_ratio = Vdata['flux_ratio'].values[0]        # flux ratio of 1.4/0.843 GHz 
    flux_143 = Vdata['flux_143'].values[0]            # 143 GHz [mJy]
    flux_xray = Vdata['flux_xray'].values[0]          # x-ray flux 0.1-2.4 keV [fW/m^2]; f = 10E-15
                                                      # F(lambda) = [W/m^2/(wavelength)]
                                                      # => [fW/m^2] = lambda*F(lambda) = nu*F(nu)
    flux_gamma = Vdata['flux_gamma'].values[0]        # fermi flux 1-100 GeV [ph/cm^2/s]
                                                      # 1 Jy = 1.51E7 ph/s/m^2 = 1.51E3 ph/s/cm^2
    Spec_index = Vdata.Spec_index[0]                  # radio-optical spectral index
    ### ACC
    flux_ALMA = Vdata['flux_ALMA'].values             # flux density [Jy]
    errflux = Vdata['errflux'].values                 # error of flux density [Jy]
    band = Vdata['band'].values  
    freq = Vdata['freq'].values                       # frequency [GHz]
    date = Vdata['date'].values   
    ### VLASS data   
    V_Fint = Vdata['Ftot'].values                    # integrated flux from 2-4 GHz [mJy]
    V_eFint = Vdata['eFtot'].values                       # error of total flux [mJy]
    V_Fp = Vdata['Fpeak'].values                     # peak flux from 2-4 GHz [mJy/beam]
    V_eFp = Vdata['eFpeak'].values                   # error of peak flux [mJy/beam]
    ### GLEAM data
    Fpwide = Gdata['Fpwide'].values                   # peak flux from 170-231 MHz [Jy/beam]
    e_Fpwide = Gdata['e_Fpwide'].values               # error of peak flux from 170-231 MHz [Jy/beam]
    Fintwide = Gdata['Fintwide'].values               # integrated flux from 170-231 MHz [Jy]
    e_Fintwide = Gdata['e_Fintwide'].values           # error of integrated flux from 170-231 MHz [Jy]
    # for all data below number of each value name is the centralfrequency of the measurement
    # frequency ranging from number of each value name +-4 [MHz]
    # eg., Fp076 => Frequecny from 76-4 ~ 76+4 = 72~80 MHz
    # eg., Fp084 => Frequecny from 84-4 ~ 84+4 = 80~88 MHz
    GFp = [None] * 20
    eGFp = [None] * 20
    GFint = [None] * 20
    eGFint = [None] * 20
    GFpname = ['Fp076', 'Fp084', 'Fp092', 'Fp099', 'Fp107', 'Fp115', 'Fp122', 'Fp130', 'Fp143', 'Fp151', 'Fp158', 'Fp166',
                'Fp174', 'Fp181', 'Fp189', 'Fp197', 'Fp204', 'Fp212', 'Fp220', 'Fp227']
    eGFpname = ['e_Fp076', 'e_Fp084', 'e_Fp092', 'e_Fp099', 'e_Fp107', 'e_Fp115', 'e_Fp122', 'e_Fp130', 'e_Fp143', 'e_Fp151', 
                'e_Fp158', 'e_Fp166', 'e_Fp174', 'e_Fp181', 'e_Fp189', 'e_Fp197', 'e_Fp204', 'e_Fp212', 'e_Fp220', 'e_Fp227']
    GFintname = ['Fint076', 'Fint084', 'Fint092', 'Fint099', 'Fint107', 'Fint115', 'Fint122', 'Fint130', 'Fint143', 'Fint151',
                 'Fint158', 'Fint166', 'Fint174', 'Fint181', 'Fint189', 'Fint197', 'Fint204', 'Fint212', 'Fint220', 'Fint227']
    eGFintname = ['e_Fint076', 'e_Fint084', 'e_Fint092', 'e_Fint099', 'e_Fint107', 'e_Fint115', 'e_Fint122', 'e_Fint130',  
                  'e_Fint143', 'e_Fint151', 'e_Fint158', 'e_Fint166', 'e_Fint174', 'e_Fint181', 'e_Fint189', 'e_Fint197', 
                  'e_Fint204', 'e_Fint212', 'e_Fint220', 'e_Fint227']

    for i in range(0, len(GFp)):
        GFp[i] = Gdata[GFpname[i]].values[ np.logical_not(np.isnan(Gdata[GFpname[i]].values)) ]
            # peak flux from 72-80 MHz [Jy/beam]
        eGFp[i] = Gdata[eGFpname[i]].values[ np.logical_not(np.isnan(Gdata[eGFpname[i]].values)) ]
            # rms uncertainty in fit for or peak flux in 072-080MHz [Jy/beam]
        GFint[i] = Gdata[GFintname[i]].values[ np.logical_not(np.isnan(Gdata[GFintname[i]].values)) ]
            # Integrated flux in 072-080MHz [Jy]
        eGFint[i] = Gdata[eGFintname[i]].values[ np.logical_not(np.isnan(Gdata[eGFintname[i]].values)) ]
            # rms uncertainty in fit for integrated flux in 072-80 MHz [Jy/beam] 
            
    ### SDSS data
    Smag = [None]*5
    eSmag = [None]*5
    Smagname = ['umag','gmag','rmag','imag','zmag']
    eSmagname = ['eumag','egmag','ermag','eimag','ezmag']
    for i in range(0, len(Smag)):
        Smag[i] = Sdata[Smagname[i]].values[ np.logical_not(np.isnan(Sdata[Smagname[i]].values)) ]
        eSmag[i] = Sdata[eSmagname[i]].values[ np.logical_not(np.isnan(Sdata[eSmagname[i]].values)) ]
                    
    ### WISE data
    Wmag = [None]*7
    eWmag = [None]*7
    Wmagname = ['W1mag','W2mag','W3mag','W4mag','Jmag','Hmag','Kmag']
    eWmagname = ['eW1mag','eW2mag','eW3mag','eW4mag','eJmag','eHmag','eKmag']
    for i in range(0, len(Wmag)):
        Wmag[i] = Wdata[Wmagname[i]].values
        eWmag[i] = Wdata[eWmagname[i]].values
        
    ### FIRST
    F_Fp= Fdata['Fpeak'].values                  # peak flux between 0.75-1.5 GHz [mJy] (750-1500 MHz)
    F_Fint = Fdata['Fint'].values                # integrated flux between 0.75-1.5 GHz [mJy]
    F_eFp = Fdata['rms_error'].values            # error of flux density [mJy]
    
    #######################################################################################################
    # set Jy2erg = 10E-23 to convert Jy to erg/s/cm^2
    # set Jy2erg = 1 to display SED flux density in the unit of Jy
    Jy2erg = 10E-23
    #######################################################################################################
    ### put peak flux and observed frequency into an array
    # BZCAT
    BZradio = [143E9, flux_143/1000*143E9*Jy2erg]
    BZxray = [3E17, flux_xray*1E-15]
    BZgamma = [1.3E25, flux_gamma*1.22E25/1.51E3*Jy2erg]  # 1 -100 GeV => median 50.5 GeV ~1.22*10^25 Hz
    # ACC
    ACCcom = [freq*1E9, flux_ALMA*freq*1E9*Jy2erg]
    # VLASS
    Vpeak = [np.linspace(3E9, 3E9, num = len(V_Fp)), V_Fp*3E9/1000*Jy2erg]  
    #Vpeak[1][np.where( Vpeak[1] < max(Vpeak[1])*0.8 )] = np.nan
    # GLEAM peak flux
    GFpdata = [None] * 20 
    for i in range(0, len(GFp)):
        Gfreq = float(GFpname[i][2::])*1E6
        GFpdata[i] = [np.linspace(Gfreq, Gfreq, num = len(GFp[i])), GFp[i]*Gfreq*Jy2erg]
    # SDSS magnitude to AB magnitude to flux density 
    Sdata = [None]*5
    Sabmag = [None]*5
    eSabmag = [None]*5
    Sflux = [None]*5
    eSflux = [None]*5
    b = [1.4E-10, 0.9E-10, 1.2E-10, 1.8E-10, 7.4E-10]
    Sfreq = [8.44E14, 6.40E14, 4.87E14, 4.01E14, 3.36E14]
    ABcorr = [-0.04, 0, 0, 0, 0.02]
    for i in range(0, len(Sdata)):
        for j in range(0, 5):
            if i == j:
                Sabmag[j] = Smag[j]+ABcorr[j] 
                eSabmag[j] = eSmag[j]+ABcorr[j] 
            
        Sflux[i] = 10**((8.9-Sabmag[i])/2.5) 
        eSflux[i] = 10**((8.9- (Sabmag[i]+eSabmag[i])) /2.5) 
        Sdata[i] = [np.linspace(Sfreq[i], Sfreq[i], num = len(Sflux[i])), Sflux[i]*Sfreq[i]*Jy2erg]
    # WISE magnitude to AB magnitude to flux density 
    Wdata = [None]*7
    Wabmag = [None]*7
    eWabmag = [None]*7
    Wflux = [None]*7
    eWflux = [None]*7
    Wfreq = [8.95E13, 6.72E13, 2.59E13, 1.36E13, 2.43E14, 1.78E14, 1.39E14]
    ABcorr = [2.699, 3.339, 5.174, 6.620, 0.91, 1.39, 1.85]
    for i in range(0, len(Wdata)):
        for j in range(0, 7):
            if i == j:
                Wabmag[j] = Wmag[j]+ABcorr[j]
                eWabmag[j] = eWmag[j]+ABcorr[j]
            
        Wflux[i] = 10**((8.9-Wabmag[i])/2.5)
        eWflux[i] = 10**( (8.9-(Wabmag[i]+eWabmag[i]))/2.5)
        Wdata[i] = [np.linspace(Wfreq[i], Wfreq[i], num = len(Wflux[i])), Wflux[i]*Wfreq[i]*Jy2erg]   
    # FIRST
    Fpeak = [np.linspace(1.125E9, 1.125E9, num = len(F_Fp)), F_Fp*1.125E9/1000*Jy2erg]

    ### put uncertainty of each data point in to an array
    # BZCAT peak flux uncertainty
    e_freq_gamma = 0.434*(1.1E25/1.3E25)
    e_freq_xray = 0.434*(2.8E17/3.0E17)
    # ACC peak flux uncertainty
    e_ACC = [0.434*( (errflux*freq*1.0E9*Jy2erg)/(flux_ALMA*freq*1.0E9*Jy2erg) )]
    # VLASS observing frequency and peak flux uncertainty
    e_freq_VLASS = [np.linspace(3E9, 3E9, num = len(V_Fp)), 0.434*(1.0E9/3.0E9)] # frequency error
    e_Vpeak = [0.434*(V_eFp*Jy2erg/V_Fp*Jy2erg)] # peak flux error
    # GLEAM observing frequency and peak flux uncertainty 
    eGFp_freq = [None]*20 
    eGFpdata = [None]*20
    for i in range(0, len(eGFpdata)):
        eGFp_freq[i] =[ np.linspace(Gfreq, Gfreq, num = len(GFp[i])), 0.434*(4/float(GFpname[i][2::]))] # freq error
        eGFpdata[i] = 0.434*(eGFp[i]*Jy2erg/GFp[i]*Jy2erg) # peak flux error
    # SDSS observing frequency and peak flux uncertainty 
    eS_freq = [None]*5
    eSdata = [None]*5
    maxfreq = [9.84E14, 7.93E14, 5.54E14, 4.48E14, 3.37E14]
    minfreq = [7.45E14, 5.41E14, 4.29E14, 3.58E14, 2.77E14]
    for i in range(0, len(eSdata)):
        eS_freq[i] =[ 0.434*( (Sfreq[i]-minfreq[i])/Sfreq[i] ), 0.434*( (maxfreq[i]-Sfreq[i])/Sfreq[i] )] 
        eSdata[i] = [0.434*((eSflux[i]*Sfreq[i]*Jy2erg-Sdata[i][1])/Sdata[i][1]), 0.434*( (eSflux[i]*Sfreq[i]*Jy2erg-Sdata[i][1])/Sdata[i][1]) ]       
    # WISE observing frequency and peak flux uncertainty 
    eW_freq = [None]*7
    eWdata = [None]*7
    maxfreq = [1.09E14, 7.57E13, 4.03E13, 1.54E13, 2.75E14, 2.06E14, 1.55E14]
    minfreq = [7.75E13, 5.62E13, 1.74E13, 1.07E13, 2.11E14, 1.50E14, 1.23E14]
    for i in range(0, len(eWdata)):
        eW_freq[i] =[ 0.434*( (Wfreq[i]-minfreq[i])/Wfreq[i] ), 0.434*( (maxfreq[i]-Wfreq[i])/Wfreq[i] )]
        eWdata[i] = [ 0.434*((eWflux[i]*Wfreq[i]*Jy2erg-Wdata[i][1])/Wdata[i][1]), 0.434*((eWflux[i]*Wfreq[i]*Jy2erg-Wdata[i][1])/Wdata[i][1])]   
    # FIRST observing frequency and peak flux uncertainty
    e_freq_FIRST = [np.linspace(1.125E9, 1.125E9, num = len(F_Fp)), 0.434*(0.375E9/1.125E9)] # frequency error
    e_Fpeak = [0.434*(F_eFp*Jy2erg/F_Fp*Jy2erg)] # peak flux error
    #######################################################################################################
    ### put integrated flux and observed frequency into an array
    # VLASS
    Vint = [np.linspace(3E9, 3E9, num = len(V_Fint)), V_Fint*3E9/1000*Jy2erg]
    # GLEAM integrated flux
    GFintdata = [None] * 20 
    for i in range(0, len(GFint)):
        Gfreq = float(GFintname[i][4::])*1E6
        GFintdata[i] = [np.linspace(Gfreq, Gfreq, num = len(GFint[i])), GFint[i]*Gfreq*Jy2erg]
    # FIRST
    Fint = [np.linspace(1.125E9, 1.125E9, num = len(F_Fint)), F_Fint*1.125E9/1000*Jy2erg]
    
    #######################################################################################################
    ### spectral index fitting
    #######################################################################################################
    ### spectral index fitting (radio to sub-mm)
    nGFpdata = [item for sublist in GFpdata for item in sublist]
    nGFpfreq = []
    nGFpflux = []
    for i in range(0, int(len(GFpdata))):
        nGFpfreq.append(nGFpdata[2*i])
        nGFpflux.append(nGFpdata[2*i+1])
        
    nWdata = [item for sublist in Wdata for item in sublist]
    nWfreq = []
    nWflux = []
    for i in range(0, int(len(Wdata))):
        nWfreq.append(nWdata[2*i])
        nWflux.append(nWdata[2*i+1])
        
    nSdata = [item for sublist in Sdata for item in sublist]
    nSfreq = []
    nSflux = []
    for i in range(0, int(len(Sdata))):
        nSfreq.append(nSdata[2*i])
        nSflux.append(nSdata[2*i+1])
        
    ne_ACC = [item for sublist in e_ACC for item in sublist]
    ne_Vpeak = [item for sublist in e_Vpeak for item in sublist]
    neGFp = [item for sublist in eGFpdata for item in sublist]
    ne_Fpeak = [item for sublist in e_Fpeak for item in sublist]
    
    # frequency data combination (BZCAT_radio, ACC, VLASS, GLEAM, FIRST)
    allfreq = []
    allfreq = np.hstack([np.array(np.log10(BZradio[0])), np.log10(ACCcom[0][0::]).flatten(), 
                         np.log10(Vpeak[0][0]).flatten(), np.log10(nGFpfreq).flatten(), 
                         np.log10(Fpeak[0][0]).flatten() ])
    # flux density combination
    allflux = []
    allflux = np.hstack([np.array(np.log10(BZradio[1])), np.array(np.log10(ACCcom[1][0::])), 
                         np.log10(sum(Vpeak[1][0::])).flatten(), np.log10(nGFpflux).flatten(), 
                         np.log10(sum(Fpeak[1][0::])).flatten() ])
    # flux density uncertainty combnation
    allfluxerr = []
    allfluxerr = np.hstack([0, ne_ACC, ne_Vpeak, neGFp, ne_Fpeak]) # 0 is error for BZCAT radio data
    # find nan and infinity index of conbined frequency and flux density
    data_ind = []
    data_ind = np.hstack( [np.where( np.isnan(allfreq) )[0].flatten(), np.where( np.isnan(allflux))[0].flatten(), 
                      np.where( np.isinf(allfreq) )[0].flatten(), np.where( np.isinf(allflux) )[0].flatten() ])
    # find nan and infinity index of conbined flux density uncertainty
    error_ind = []
    error_ind = np.hstack( [np.where( np.isnan(allfluxerr) )[0].flatten(), np.where( np.isinf(allfluxerr) )[0].flatten() ])
    
    xx = np.delete(allfreq, data_ind)
    yy = np.delete(allflux, data_ind)
    yyerr = np.delete(allfluxerr, error_ind)
    
    # define fitting function
    def func(x, a, c):
        return x**(a)+c
        
    if len(xx) >= 2:
        p0 = [min(xx), min(yy)]
        popt, pcov = curve_fit(func, xx, yy, p0 = p0) # p0 is initial point
        yfit = func(xx, *popt)
    
    alpha = '%7.5f' % float(popt[0])
    
    #######################################################################################################
    ### spectral index fitting (radio ONLY)    
    # frequency data combination (VLASS, GLEAM, FIRST)
    allfreqRad = []
    allfreqRad = np.hstack([np.log10(Vpeak[0][0]).flatten(), np.log10(nGFpfreq).flatten(), 
                         np.log10(Fpeak[0][0]).flatten() ])
    # flux density combination
    allfluxRad = []
    allfluxRad = np.hstack([np.log10(sum(Vpeak[1][0::])).flatten(), np.log10(nGFpflux).flatten(), 
                         np.log10(sum(Fpeak[1][0::])).flatten() ])
    # flux density uncertainty combnation
    allfluxerrRad = []
    allfluxerrRad = np.hstack([ne_Vpeak, neGFp, ne_Fpeak]) # 0 is error for BZCAT radio data
    # find nan and infinity index of conbined frequency
    data_indRad = []
    data_indRad = np.hstack( [np.where( np.isnan(allfreqRad) )[0].flatten(), np.where( np.isnan(allfluxRad))[0].flatten(), 
                      np.where( np.isinf(allfreqRad) )[0].flatten(), np.where( np.isinf(allfluxRad) )[0].flatten() ])
    # find nan and infinity index of conbined flux density
    error_indRad = []
    error_indRad = np.hstack( [np.where( np.isnan(allfluxerrRad) )[0].flatten(), np.where( np.isinf(allfluxerrRad) )[0].flatten() ])
    
    xxRad = np.delete(allfreqRad, data_indRad)
    yyRad = np.delete(allfluxRad, data_indRad)
    yyerrRad = np.delete(allfluxerrRad, error_indRad)
    
    if len(xxRad) >= 2:
        # define fitting function   
        def func(x, a, c):
            return x**(a)+c
        
        p0 = [min(xxRad), min(yyRad)]
        poptRad, pcovRad = curve_fit(func, xxRad, yyRad, p0=p0)
        yfitRad = func(xxRad, *poptRad)
    
        alphaRad = '%7.5f' % float(poptRad[0])
    else: 
        alphaRad = np.nan
        
    #######################################################################################################
    ### K-correction (using radio to mm and sub-mm spectral index)
    # frequency data combination (BZCAT_radio, ACC, VLASS, GLEAM, FIRST)
    allfreq = []
    allfreq = np.hstack([np.array(np.log10(BZradio[0])), np.log10(ACCcom[0][0::]).flatten(), np.log10(Vpeak[0][0]).flatten(), 
                         np.log10(nGFpfreq).flatten(), np.log10(Fpeak[0][0]).flatten() ])
    # flux density combination
    allflux = []
    allflux = np.hstack([np.array(np.log10(BZradio[1])), np.array(np.log10(ACCcom[1][0::])), np.log10(sum(Vpeak[1][0::])).flatten(), 
                         np.log10(nGFpflux).flatten(), np.log10(sum(Fpeak[1][0::])).flatten() ])
    
    data_ind = []
    data_ind = np.hstack( [np.where( np.isnan(allfreq) )[0].flatten(), np.where( np.isnan(allflux))[0].flatten(), 
                      np.where( np.isinf(allfreq) )[0].flatten(), np.where( np.isinf(allflux) )[0].flatten() ])
    
    xxx = np.delete(allfreq, data_ind)
    yyy = np.delete(allflux, data_ind)
    
    xxK = xxx*(1+z)
    yyK = yyy*(1+z)**(float(alpha)-1)
    
    def func(x, a, c):
        return x**(a)+c
        
    if len(xxK) >= 2:
        p0 = [min(xxK), min(yyK)]
        poptK, pcov = curve_fit(func, xxK, yyK, maxfev = 1000) # p0 is initial point
        yfitK = func(xxK, *poptK)
        
        alphaK = '%7.5f' % float(poptK[0])
    
    ### K-correction (using radio spectral index ONLY) 
    # frequency data combination (VLASS, GLEAM, FIRST)
    allfreqRad = []
    allfreqRad = np.hstack([np.log10(Vpeak[0][0]).flatten(), np.log10(nGFpfreq).flatten(), np.log10(Fpeak[0][0]).flatten()])
    #flux density combination
    allfluxRad = []
    allfluxRad = np.hstack([np.log10(sum(Vpeak[1][0::])).flatten(), np.log10(nGFpflux).flatten(), np.log10(sum(Fpeak[1][0::])).flatten() ])
    
    data_indRad = []
    data_indRad = np.hstack( [np.where( np.isnan(allfreqRad) )[0].flatten(), np.where( np.isnan(allfluxRad))[0].flatten(), 
                      np.where( np.isinf(allfreqRad) )[0].flatten(), np.where( np.isinf(allfluxRad) )[0].flatten() ])
    
    xxxRad = np.delete(allfreqRad, data_indRad)
    yyyRad = np.delete(allfluxRad, data_indRad)
    yyerrRad = np.delete(allfluxerrRad, error_indRad)
    
    xxRadK = xxxRad*(1+z)
    yyRadK = yyyRad*(1+z)**(float(alphaRad)-1)
    
    def func(x, a, c):
        return x**(a)+c
        
    if len(xxRadK) >= 2:
        poptRadK, pcov = curve_fit(func, xxRadK, yyRadK, maxfev = 1000)
        yfitRadK = func(xxRadK, *poptRadK)
    
        alphaRadK = '%7.5f' % float(poptRadK[0])
    else:
        alphaRadK = np.nan
        
    #######################################################################################################
    ### plot SED figure 
    if (plotfigure == 1): 
        # str(Fpeak[1][0]) != str('nan'), plotfigure == 1, 
        # str(np.log10(Wdata[0][0][1])) != str('nan') and str(np.log10(Wdata[0][0][1]))) != str('-inf')
        plt.rcParams['figure.figsize'] = [18, 9]
        plt.rcParams['axes.linewidth'] = 3
        plt.rc('font', size = 16)    
        mpl.rcParams['xtick.major.size'] = 6
        mpl.rcParams['xtick.major.width'] = 2
        mpl.rcParams['ytick.major.size'] = 6
        mpl.rcParams['ytick.major.width'] = 2
        fig = plt.figure()
        gs = fig.add_gridspec(1, 2, wspace = 0)
        axs = gs.subplots() # , sharey='row'
        
        ### plot peak flux 
        axs[0].plot(np.log10(GFpdata[0][0]), np.log10(GFpdata[0][1]), '.', c = "red", markersize = 6, label = 'GLEAM')
        for i in range(0, len(GFp)):
            axs[0].plot(np.log10(GFpdata[i][0]), np.log10(GFpdata[i][1]), '.', c = "red", markersize = 6)
        axs[0].plot(np.log10(Fpeak[0][0]), np.log10(sum(Fpeak[1][0::])), '.', c = "red", markersize = 6, label = 'FIRST ($\\nu$~9.05)')
        axs[0].plot(np.log10(Vpeak[0][0]), np.log10(sum(Vpeak[1][0::])), '.', c = "red", markersize = 6, label = 'VLASS ($\\nu$~9.48)')
        axs[0].plot(np.log10(ACCcom[0][0::]), np.log10(ACCcom[1][0::]), '.', c = "gray", markersize = 6, label = 'ACC')
        axs[0].plot(np.log10(BZradio[0]), np.log10(BZradio[1]), 'x', c = "black", markersize = 6, label = 'BZCAT radio')
        # WISE
        axs[0].plot(np.log10(Wdata[0][0][0]), np.log10(Wdata[0][0][1]), '.', c = "green", markersize = 6, label = 'WISE (IR)')
        for i in range(0, len(Wmag)): #len(Wmag)
            axs[0].plot(np.log10(Wdata[i][0][0]), np.log10(Wdata[i][1][0]), '.', c = "green", markersize = 6)
        # SDSS
        axs[0].plot(np.log10(Sdata[0][0]), np.log10(Sdata[0][1]), '.', c = "blue", markersize = 6, label = 'SDSS (NIR & optical)')
        for i in range(0, len(Smag)):
            axs[0].plot(np.log10(Sdata[i][0]), np.log10(Sdata[i][1]), '.', c = "blue", markersize = 6)
        axs[0].plot(np.log10(BZxray[0]), np.log10(BZxray[1]), '.', c = "purple", markersize = 6, label = 'BZCAT x-ray')
        axs[0].plot(np.log10(BZgamma[0]), np.log10(BZgamma[1]), '.', c = "purple", markersize = 6, label = 'BZCAT gamma ray')
        
        ### plot integrated flux
        # VLASS int
        #ax1.plot(np.log10(Vint[0][0::]), np.log10(Vint[1][0::]), '.', c = "black", markersize = 6, label = 'VLASS int')
        # GLEAM int
        #ax1.plot(np.log10(GFintdata[0][0]), np.log10(GFintdata[0][1]), '.', c = "black", markersize = 6, label = 'GLEAM int')
        #for i in range(0, len(GFint)):
        #    ax1.plot(np.log10(GFintdata[i][0]), np.log10(GFintdata[i][1]), '.', c = "black", markersize = 6)
        # FIRST int
        #ax1.plot(np.log10(Fint[0][0::]), np.log10(Fint[1][0::]), '.', c = "black", markersize = 6, label = 'FIRST int')
            
        ### plot uncertainty
        # Error of BZCAT (x-ray, gamma-ray)
        axs[0].errorbar(np.log10(BZgamma[0]), np.log10(BZgamma[1]), xerr = e_freq_gamma, ecolor = "purple")
        axs[0].errorbar(np.log10(BZxray[0]), np.log10(BZxray[1]), xerr = e_freq_xray, ecolor = "purple")
        # Error of ACC
        axs[0].errorbar(np.log10(ACCcom[0][0::]), np.log10(ACCcom[1][0::]), yerr = ne_ACC, ecolor = "gray", linestyle = '')
        # Error of VLASS 
        #plt.errorbar(np.log10(Vpeak[0][0::]), np.log10(Vpeak[1][0::]), xerr = e_freq_VLASS[1], ecolor = "red", linestyle = '')
        #for i in range(0, len(e_Vpeak)):
        #    plt.errorbar(np.log10(Vpeak[0][0::]), np.log10(Vpeak[1][0::]), yerr = e_Vpeak[i], ecolor = "red", linestyle = '')
        # Error of GLEAM observing frequency and flux density
        for i in range(0, len(eGFpdata)):
            axs[0].errorbar(np.log10(GFpdata[i][0]), np.log10(GFpdata[i][1]), xerr = eGFp_freq[i][1] , yerr = eGFpdata[i], ecolor = "red", linestyle = '')
        # Error of SDSS observing frequency and flux density
        for i in range(0, len(eSdata)):
            axs[0].errorbar(np.log10(Sdata[i][0]), np.log10(Sdata[i][1]), xerr = eS_freq[i][1] , yerr = eSdata[i], ecolor = "blue", linestyle = '')
        # Error of WISE observing frequency and flux density
        for i in range(0, len(eWdata)): #
            axs[0].errorbar(np.log10(Wdata[i][0][0]), np.log10(Wdata[i][1][0]), xerr = eW_freq[i][1], yerr = eWdata[i][0][0], ecolor = "green", linestyle = '')
        # Error of FIRST
        #axs[0].errorbar(np.log10(Fpeak[0][0::]), np.log10(Fpeak[1][0::]), xerr = e_freq_FIRST[1], ecolor = "red", linestyle = '')
        #for i in range(0, len(e_Fpeak)):
        #    axs[0].errorbar(np.log10(Fpeak[0][0::]), np.log10(Fpeak[1][0::]), yerr = e_Fpeak[i], ecolor = "red", linestyle = '')
        
        axs[0].grid(True)        
        axs[0].set_xlim (7, 28) #7, 28
        axs[0].set_ylim (-20, 0) # -20, 0

        # formatting labels
        #fig.autofmt_xdate()
        #######################################################################################################
        ### plot spectral index fitting
        # plot radio to sub-mm spectral index
        if len(xx) >= 2:
            axs[1].plot(xx, yfit, '-', c = "red", label='Spectral index (radio to sub-mm)\n after K-correction: a=%5.3f' % float(popt[0]))
        # plot radio spectral index
        if len(xxRad) >= 2:
            axs[1].plot(xxRad, yfitRad, '-', c = "blue", label='Spectral index \n(radio only): a=%5.3f' % float(poptRad[0]))
        # plot radio to sub-mm spectral index after K-correction
        if len(xxK) >= 2:
            axs[1].plot(xxK, yfitK, '-', c = "orange",label='K-corrections spectral index \n(radio to sub-mm): a=%5.3f' % float(poptK[0]))
        # plot radio spectral index after K-correction
        if len(xxRadK) >= 2:
            axs[1].plot(xxRadK, yfitRadK, '-', c = "green",label='K-corrections spectral index \n(radio to sub-mm): a=%5.3f' % float(poptK[0]))
        ### check flux used for spectral index fitting    
        axs[1].plot(xx, yy, '*', c = 'red') # radio to mm and sub-mm
        axs[1].plot(xxRad, yyRad, '*', c = 'blue') # radio ONLY
        axs[1].plot(xxK, yyK, '*', c = 'orange') # radio to mm and sub-mm after z correction
        axs[1].plot(xxRadK, yyRadK, '*', c = 'green') # radio ONLY after z correction
    
        axs[1].grid(True)
        maxnum = '%2.0f' % float(max(xxK)+5)
        axs[1].set_xlim (7, float(maxnum))
        axs[1].set_ylim (-20, 0)
        fig.legend(loc='upper right', bbox_to_anchor = (1.1, 0.9), shadow=True, fontsize = 15, markerscale = 2) 
        
        for ax in axs.flat:
            ax.set(xlabel='log($\\nu$) [Hz]', ylabel='log($\\nu$ F($\\nu$)) [erg $cm^{-2} s^{-1}$]')
            
        for ax in axs.flat:
            ax.label_outer()
        
        fig.suptitle('SED of '+ Name+' (z = '+str(z)+')', fontsize = 25)

        plotname = '/Users/87steven/Documents/ASIAA/SED progress/SED of all sources/SED of '+ Name
        fig.savefig(plotname, dpi = 300, bbox_inches='tight')
    
    print('================================================ Figure saved succfully ================================================')

    df['Name'].append(Name)
    df['RA'].append(RA)
    df['DEC'].append(DEC)
    df['z'].append(z)
    df['class_name'].append(cla)
    df['alpha'].append(alpha)
    df['alphaRad'].append(alphaRad)
    df['alphaK'].append(alphaK)
    df['alphaRadK'].append(alphaRadK)
    
CSVfile = f'/Users/87steven/Documents/ASIAA/SED progress/spectral index.csv'   
dff = pd.DataFrame(df)      
dff.to_csv(CSVfile, index = False)

In [32]:
    ### K-correction (using radio spectral index ONLY) 
    # frequency data combination (VLASS, GLEAM, FIRST)
    allfreqRad = []
    allfreqRad = np.hstack([np.log10(Vpeak[0][0]).flatten(), np.log10(nGFpfreq).flatten(), 
                         np.log10(Fpeak[0][0]).flatten() ])
    #flux density combination
    allfluxRad = []
    allfluxRad = np.hstack([np.log10(sum(Vpeak[1][0::])).flatten(), np.log10(nGFpflux).flatten(), 
                         np.log10(sum(Fpeak[1][0::])).flatten() ])
    
    data_indRad = []
    data_indRad = np.hstack( [np.where( np.isnan(allfreqRad) )[0].flatten(), np.where( np.isnan(allfluxRad))[0].flatten(), 
                      np.where( np.isinf(allfreqRad) )[0].flatten(), np.where( np.isinf(allfluxRad) )[0].flatten() ])
    
    xxxRad = np.delete(allfreqRad, data_indRad)
    yyyRad = np.delete(allfluxRad, data_indRad)
    yyerrRad = np.delete(allfluxerrRad, error_indRad)
    
    xxRadK = xxxRad*(1+z)
    yyRadK = yyyRad*(1+z)**(float(alphaRad)-1)
    
    def func(x, a, c):
        return x**(a)+c
        
    if len(xxRadK) >= 2:
        poptRadK, pcov = curve_fit(func, xxRadK, yyRadK, maxfev = 1000)
        yfitRadK = func(xxRadK, *poptRadK)
    
        alphaRadK = '%7.5f' % float(poptRadK[0])
    else:
        alphaRadK = np.nan
        

In [ ]:
        plt.rcParams['figure.figsize'] = [18, 9]
        plt.rcParams['axes.linewidth'] = 3
        plt.rc('font', size = 16)    
        mpl.rcParams['xtick.major.size'] = 6
        mpl.rcParams['xtick.major.width'] = 2
        mpl.rcParams['ytick.major.size'] = 6
        mpl.rcParams['ytick.major.width'] = 2
        fig = plt.figure()
        gs = fig.add_gridspec(1, 2, wspace = 0)
        axs = gs.subplots() # , sharey='row'
        
        ### plot spectral index fitting
        # plot radio spectral index after K-correction
        if len(xxRadK) >= 2:
            axs[1].plot(xxRadK, yfitRadK, '-', c = "green",label='K-corrections spectral index \n(radio to sub-mm): a=%5.3f' % float(poptK[0]))
       
        axs[1].plot(xxRadK, yyRadK, '*', c = 'green') # radio ONLY after z correction